# tsgettoolbox and tstoolbox
## 'tsgettoolbox nwis ...': Download data from the National Water Information System (NWIS)
This notebook is to illustrate the command line usage for 'tsgettoolbox' and 'tstoolbox' to download and work with data from the National Water Information System (NWIS).  There is a different notebook to do the same thing from within a Python program called tsgettoolbox-nwis-api.

First off, always nice to remind myself about the options.  Each sub-command has their own options kept consistent with the options available from the source service.  The way that NWIS works is you have one major filter and one or more minor filters to define what sites you want.

In [13]:
tsgettoolbox nwis --help

usage: tsgettoolbox nwis [-h] [--sites <str>] [--stateCd <str>] [--huc <str>]
  [--bBox <str>] [--countyCd <str>] [--parameterCd <str>] [--period <str>]
  [--startDT <str>] [--endDT <str>] [--siteType <str>] [--modifiedSince <str>]
  [--agencyCd <str>] [--siteStatus <str>] [--altMin <float>] [--altMax
  <float>] [--drainAreaMin <float>] [--drainAreaMax <float>] [--aquiferCd
  <str>] [--localAquiferCd <str>] [--wellDepthMin <float>] [--wellDepthMax
  <float>] [--holeDepthMin <float>] [--holeDepthMax <float>] [--database
  <str>] [--statReportType <str>] [--statType <str>] [--missingData <str>]
  [--statYearType <str>]

There are three main NWIS databases. The 'tsgettoolbox' can currently pull from
the Instantaneous Value database (--database-iv) for sub-daily interval data
starting in 2007, the Daily Values database (--database=dv), or the Statistics
database for daily/monthly/annual statistics. Detailed documnetation is
available at <http://waterdata.usgs.gov/nwis>.

Site local time is

Let's say that I want flow (parameterCd=00060) for site '02325000'.  I first make sure that I am getting what I want by allowing the output to be printed to the screen.  Note the pipe ('|') that directs output to the 'head' command to display the top 10 lines of the time-series.

In [14]:
tsgettoolbox nwis --sites 02325000 --startDT '2000-01-01' --parameterCd 00060 | head

Datetime,USGS-02325000-00060
2000-01-01,82
2000-01-02,81
2000-01-03,80
2000-01-04,79
2000-01-05,75
2000-01-06,75
2000-01-07,74
2000-01-08,73
2000-01-09,75


Then I redirect to a file with "> filename.csv" so that I don't have to wait for the USGS NWIS services for the remaining work or analysis.

In [15]:
tsgettoolbox nwis --sites 02325000 --startDT '2000-01-01' --parameterCd 00060 > 02325000_flow.csv

## 'tstoolbox ...': Process data using 'tstoolbox'
Now lets use "tstoolbox" to plot the time-series.  Note the redirection again, this time for input as "< filename.csv".  Default plot filename is "plot.png".

In [16]:
tstoolbox plot < 02325000_flow.csv

![title](plot.png)

'tstoolbox plot' has many options that can be used to modify the plot.

In [17]:
tstoolbox plot --help

usage: tstoolbox plot [-h] [--ofilename <str>] [--type <str>] [--xtitle <str>]
  [--ytitle <str>] [--title <str>] [--figsize <str>] [--legend LEGEND]
  [--legend_names <str>] [--subplots] [--sharex] [--sharey] [--style <str>]
  [--logx] [--logy] [--xaxis <str>] [--yaxis <str>] [--xlim XLIM] [--ylim
  YLIM] [--secondary_y] [--mark_right] [--scatter_matrix_diagonal <str>]
  [--bootstrap_size BOOTSTRAP_SIZE] [--bootstrap_samples BOOTSTRAP_SAMPLES]
  [--norm_xaxis] [--norm_yaxis] [--lognorm_xaxis] [--lognorm_yaxis]
  [--xy_match_line <str>] [--grid GRID] [-i <str>] [-s <str>] [-e <str>]
  [--label_rotation <int>] [--label_skip <int>] [--force_freq FORCE_FREQ]
  [--drawstyle <str>] [--por] [--columns COLUMNS] [--invert_xaxis]
  [--invert_yaxis] [--plotting_position <str>]

Plot data.

optional arguments:
  -h | --help
      show this help message and exit
  --ofilename <str>
      Output filename for the plot. Extension defines the type, ('.png').
      Defaults to 'plot.png'.
  --type <str

In [18]:
tstoolbox plot --ofilename flow.png --ytitle 'Flow (cfs)' --title '02325000: FENHOLLOWAY RIVER NEAR PERRY, FLA' --legend False < 02325000_flow.csv

![title](flow.png)

## Monthly Average Flow
You can also use tstoolbox to make calculations on the time-series, for example to aggregate to monthly average flow:

In [21]:
tstoolbox aggregate --agg_interval M --statistic mean < 02325000_flow.csv | head

Datetime,USGS-02325000-00060_mean
2000-01-31,80
2000-02-29,89.7931
2000-03-31,80.0323
2000-04-30,81.7667
2000-05-31,90.8387
2000-06-30,94.4
2000-07-31,85.9032
2000-08-31,83.0323
2000-09-30,128.067


In [20]:
tstoolbox aggregate --agg_interval M --statistic mean < 02325000_flow.csv | tstoolbox plot --ofilename plot_monthly.png --drawstyle steps-pre

![title](plot_monthly.png)